### マルウェアのダウンロード
https://github.com/InQuest/malware-samples/blob/master/2019-02-Trickbot/374ef83de2b254c4970b830bb93a1dd79955945d24b824a0b35636e14355fe05?raw=true  
を取得して同じディレクトリに格納し、「374ef83de2b254c4970b830bb93a1dd79955945d24b824a0b35636e14355fe05」という名前で保存する。  
マルウェアのファイルなので取り扱いは要注意、自己責任で。  

In [ ]:
from tensorflow.python .keras.models import load_model
from sklearn.preprocessing import StandardScaler
import numpy as np
import ember
import pandas as pd

np.random.seed(0)

### モデル読み込み

In [ ]:
estimator = load_model('detect_malware_model.h5')

### 訓練データ読み込み及びスケーラーの再作成

In [ ]:
X_train, y_train, X_test, y_test = ember.read_vectorized_features("./ember_data/ember2018/")

train_rows = (y_train != -1)
X_train = X_train[train_rows]
y_train = y_train[train_rows]

# メモリ容量の問題の為、データ数を絞る。
target_num = 160000
target_index = np.random.choice(X_train.shape[0], target_num, replace=False)
X_train = X_train[target_index]
y_train = y_train[target_index]
print(pd.DataFrame(y_train).value_counts())

scaler  = StandardScaler()
X_train = scaler.fit_transform(X_train)

### emberの各種不整合の修正
ember\features.py:100 の  
output = np.zeros((16, 16), dtype=np.int)  
を
output = np.zeros((16, 16), dtype=np.int64)  
に置き換え  

ember\features.py:557 の  
lief_errors = (lief.bad_format, lief.bad_file, lief.pe_error, lief.parser_error, lief.read_out_of_bound,
                       RuntimeError)  
を  
lief_errors = (Exception)
に置き換え

### 通常ファイルの確認

In [ ]:
sample_data = open('pe', 'rb').read()

extractor = ember.PEFeatureExtractor(2)
sample_data = np.array(
    extractor.feature_vector(sample_data),
    dtype=np.float32
).reshape(1, -1)
sample_data = scaler.transform(sample_data)

probability = estimator.predict(sample_data)
pred = estimator.predict_classes(sample_data)

print("確率:", probability)
if pred:
    print("Malware!")
else:
    print("Benign file!")

### マルウェアの検出
(「Unable to find the section associated with BOUND_IMPORT」メッセージ有、入力の特徴量で上手く取れていないものがある?)

In [ ]:
sample_data = open('374ef83de2b254c4970b830bb93a1dd79955945d24b824a0b35636e14355fe05', 'rb').read()

extractor = ember.PEFeatureExtractor(2)
sample_data = np.array(
    extractor.feature_vector(sample_data),
    dtype=np.float32
).reshape(1, -1)
sample_data = scaler.transform(sample_data)

probability = estimator.predict(sample_data)
pred = estimator.predict_classes(sample_data)

print("確率:", probability)
if pred:
    print("Malware!")
else:
    print("Benign file!")